In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import csv

# Importing SKLearn clssifiers and libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [21]:
df_train = pd.read_csv('train.csv')

In [ ]:
df_train.head()

In [23]:
df_train.shape

(891, 12)

In [24]:
df_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

#### Removing redundant columns

In [25]:
df_train = df_train.drop('Name', axis=1,)
df_train = df_train.drop('Ticket', axis=1,)
df_train = df_train.drop('Cabin', axis=1,)


In [26]:
df_train = df_train.drop('PassengerId', axis=1,)

In [27]:
df_train['Family'] = df_train['SibSp'] + df_train['Parch'] + 1

In [28]:
df_train = df_train.drop('SibSp', axis=1,)
df_train = df_train.drop('Parch', axis=1,)

In [29]:
df_train["Age"].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

### Cleaning

In [30]:
features = list(df_train.columns.values)

for col in features:
    print (col,": ",sum(pd.isnull(df_train[col])))

Survived :  0
Pclass :  0
Sex :  0
Age :  177
Fare :  0
Embarked :  2
Family :  0


In [31]:
df_train["Age"] = df_train["Age"].fillna(df_train["Age"].median())

In [32]:
df_train["Embarked"].mode()

0    S
dtype: object

In [33]:
df_train["Embarked"] = df_train["Embarked"].fillna("S")

In [34]:
features = list(df_train.columns.values)

for col in features:
    print (col,": ",sum(pd.isnull(df_train[col])))

Survived :  0
Pclass :  0
Sex :  0
Age :  0
Fare :  0
Embarked :  0
Family :  0


In [35]:
df_train["Adult"] = 0

In [36]:
df_train["Adult"][df_train["Age"] >= 18] = 1

<ipython-input-36-845fd0f8fcb2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["Adult"][df_train["Age"] >= 18] = 1


In [37]:
df_train

,Survived,Pclass,Sex,Age,Fare,Embarked,Family,Adult
0,0,3,male,22.0,7.2500,S,2,1
1,1,1,female,38.0,71.2833,C,2,1
2,1,3,female,26.0,7.9250,S,1,1
3,1,1,female,35.0,53.1000,S,2,1
4,0,3,male,35.0,8.0500,S,1,1
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,13.0000,S,1,1
887,1,1,female,19.0,30.0000,S,1,1
888,0,3,female,28.0,23.4500,S,4,1
889,1,1,male,26.0,30.0000,C,1,1


In [38]:
print ("Passengers more than 18 yrs old: ",str(len(df_train[df_train["Age"] >= 18])))

print ("Number of Adults: ",str(len(df_train[df_train["Adult"] >= 1])))

Passengers more than 18 yrs old:  778
Number of Adults:  778


In [39]:
df_train = df_train.drop('Age', axis=1,)

In [40]:
df_train

,Survived,Pclass,Sex,Fare,Embarked,Family,Adult
0,0,3,male,7.2500,S,2,1
1,1,1,female,71.2833,C,2,1
2,1,3,female,7.9250,S,1,1
3,1,1,female,53.1000,S,2,1
4,0,3,male,8.0500,S,1,1
...,...,...,...,...,...,...,...
886,0,2,male,13.0000,S,1,1
887,1,1,female,30.0000,S,1,1
888,0,3,female,23.4500,S,4,1
889,1,1,male,30.0000,C,1,1


In [43]:
df1 = df_train.filter(['Pclass','Sex','Embarked','Family','Adult'], axis=1)

X = df1

In [44]:
df2 = df_train['Survived']

y = df2

In [47]:
X["Sex"][df_train["Sex"] == "male"] = 1
X["Sex"][df_train["Sex"] == "female"] = 2

<ipython-input-47-7b5a1b0d2fdc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Sex"][df_train["Sex"] == "male"] = 1
<ipython-input-47-7b5a1b0d2fdc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Sex"][df_train["Sex"] == "female"] = 2


In [48]:
X["Embarked"][df_train["Embarked"] == "S"] = 1
X["Embarked"][df_train["Embarked"] == "C"] = 2
X["Embarked"][df_train["Embarked"] == "Q"] = 3

<ipython-input-48-c2939fff091e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Embarked"][df_train["Embarked"] == "S"] = 1
<ipython-input-48-c2939fff091e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Embarked"][df_train["Embarked"] == "C"] = 2
<ipython-input-48-c2939fff091e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Embarked"][df_train["Embarked"] == "Q"] = 3


In [52]:
features_train, features_test, labels_train, labels_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [53]:
features_train.shape

(623, 5)

In [85]:
features_test.shape

(268, 5)

In [54]:
features_test

,Pclass,Sex,Embarked,Family,Adult
707,1,1,1,1,1
37,3,1,1,1,1
615,2,2,1,4,1
169,3,1,1,1,1
68,3,2,1,7,0
...,...,...,...,...,...
842,1,2,2,1,1
671,1,1,1,2,1
696,3,1,1,1,1
42,3,1,2,1,1


In [55]:
labels_train.shape

(623,)

In [88]:
labels_test.shape

(268,)

In [59]:
knn = KNeighborsClassifier( )
k_range = list(range(1,10))
weights_options = ['uniform','distance']
k_grid = dict(n_neighbors=k_range, weights = weights_options)
grid = GridSearchCV(knn, k_grid, cv=10, scoring = 'precision')
grid.fit(features_train, labels_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'weights': ['uniform', 'distance']},
             scoring='precision')

In [60]:
grid

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'weights': ['uniform', 'distance']},
             scoring='precision')

In [61]:
grid.best_params_

{'n_neighbors': 2, 'weights': 'uniform'}

#### Predict

In [62]:
label_pred = grid.predict(features_test)

In [63]:
label_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1], dtype=int64)

In [64]:
acc_clf = metrics.accuracy_score(labels_test,label_pred)
print ("classifier's accuracy: ",str(acc_clf) )

classifier's accuracy:  0.7723880597014925


In [96]:
# scr_clf = precision_recall_fscore_support(labels_test,label_pred, average='weighted')

# print ("classifier's precision: ",str(scr_clf[0])) 
# print ("classifier's recall: ",str(scr_clf[1])) 
# print ("classifier's fbeta_score: ",str(scr_clf[2])) 